# Simple channel flow using the Incompressible Solver

## What's new

 - running a simulation with the incompressible Navier-Stokes solver
 - how to use the immersed boundary method
 - post-processing, i.e. plotting and checking physical values




## Prerequisites


- the **BoSSS** framework
- a visualization tool, e.g Paraview or VisIt
- the knowledge of how to setup a database for **BoSSS**


 This tutorial will explain the basic features of the incompressible 
 Navier-Stokes solver in the **BoSSS** framework. 

 First, the simple testcase of a 2D channel flow will be explained. After 
 that, there will be a short part about the immersed boundary feature of our incompressible flow solver. 

 Therefore the flow around a cylinder will be investigated using the immersed boundary method.

 Note that **BoSSS**, at the present time contains no stand-alone single-phase 
 solver that is fully recomended - although there are some legacy solvers, e.g. SIMPLE.
 Instead, the two-phase-solver with immersed boundary is used, where the two-phase option ist deactivated.



## Problem statement

The flow is described by the unsteady Navier-Stokes equations in the fluid region

>$$ \rho_f\left(\frac{\partial \vec{u}}{\partial t}+ \vec{u} \cdot \nabla \vec{u}\right) +\nabla p - \mu_f \Delta \vec{u} = \vec{f}  $$

 and the continuity equation
>$$ \nabla \cdot \vec{u} = 0 \quad \forall\ t \in (0,T)\quad \textrm{in}\ \Omega $$

 In the equations above 
 - $\vec{u}$ is the velocity vector 
 - $p$ the pressure. 
 - The fluid density is denoted by $\rho_f$
 - $\mu_f=\rho_f \cdot \nu_f$ is the dynamic viscosity of the fluid.



## Channel

First, we initialize the new worksheet;
Note: 
1. This tutorial can be found in the source code repository as as `channel.ipynb`. 
   One can directly load this into Jupyter to interactively work with the following code examples.
2. **In the following line, the reference to `BoSSSpad.dll` is required**. 
   You must either set `#r "BoSSSpad.dll"` to something which is appropirate for your computer
   (e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if you installed the binary distribution),
   or, if you are working with the source code, you must compile `BoSSSpad` and put it side-by-side to this worksheet file
   (from the original location in the repository, you can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).

In [1]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.Diagnostics;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


The below script needs to be able to find the current output cell; this is an easy method to get it.

Now, a new database has to be created.
In this worksheet, we use a temporary database which will be deleted
after the worksheet has been executed.
For your calculation, you might consider some non-temporary alternative,
>cf. *OpenOrCreateDatabase* or *OpenOrCreateDefaultDatabase*:

In [2]:

var myDb = CreateTempDatabase();

Creating database 'C:\Users\flori\AppData\Local\Temp\1950674969'.


Create a new control object for setting up the simulation:

In [3]:
var c = new XNSE_Control();

### How to define/change input data

First, the DG polynomial degree is set:
(degree 2 for velocity and 1 for pressure).

In [4]:
c.SetDGdegree(2);

Domain and Grid variables are set (i.e. we get a channel with length 22 and height 4.1)

In [5]:
double xMin        = -2; 
double xMax        = 20;
double yMin        = -2; 
double yMax        = 2.1;
int numberOfCellsX = 44; 
int numberOfCellsY = 8;

Basic database options 

In [6]:
c.SetDatabase(myDb);
c.savetodb   = true;
c.saveperiod = 1;

Setting some variables for database saving. Here it is also possible to define tags which can be helpful for finding a particular simulation in the **BoSSS** database

In [7]:
string sessionName   = "dt = 1E20_" + numberOfCellsX + "x" + numberOfCellsY + "_k2";
c.SessionName        = sessionName;
c.ProjectDescription = "Incompressible Solver Examples";
c.Tags.Add("numberOfCellsX_" + numberOfCellsX);
c.Tags.Add("numberOfCellsY_" + numberOfCellsY);
c.Tags.Add("k2");

The grid is generated using the previously defined parameters.

In [8]:
c.GridFunc       = null; 
var xNodes       = GenericBlas.Linspace(xMin, xMax , numberOfCellsX);
var yNodes       = GenericBlas.Linspace(yMin, yMax, numberOfCellsY);
GridCommons grid = Grid2D.Cartesian2DGrid(xNodes, yNodes, CellType.Square_Linear, false);

Set the geometric location of boundary conditions by edge tags; Later we will assign values depending on these tags.

Edges that get assigned "0" are "inner edges".

In [9]:
grid.DefineEdgeTags(delegate (double[] X) {
    if (Math.Abs(X[1] - (-2)) <= 1.0e-8)
        return "wall"; // wall at y = -2
    if (Math.Abs(X[1] - (+2.1 )) <= 1.0e-8)
        return "wall"; // wall at y = +2.1
    if (Math.Abs(X[0] - (-2)) <= 1.0e-8)
        return "Velocity_Inlet"; // velocity inlet at x = -2
    if (Math.Abs(X[0] - (+20.0)) <= 1.0e-8)
        return "Pressure_Outlet"; // pressure outlet at x = +20
    throw new ArgumentException("unexpected domain boundary"); 
});

Grid Edge Tags changed.


Save the grid in the database so that the simulation can use it

In [10]:
grid

EdgeTagNames PredefinedGridPartitioning Description StorageGuid BcCellsStorageGuid PeriodicTrafo InversePeriodicTrafo NoOfUpdateCells NoOfBcCells NumberOfCells_l NumberOfBcCells RefElements EdgeRefElements SpatialDimension CellPartitioning BcCellPartitioning MyRank Size NodePartitioning iGridData .. [ [0, inner edge], [1, Velocity_Inlet], [2, wall], [3, Pressure_Outlet] ] [ ] <null> 00000000-0000-0000-0000-000000000000 00000000-0000-0000-0000-000000000000 [ ] [ ] 301 0 301 0 [ BoSSS.Foundation.Grid.RefElements.Square ] [ BoSSS.Foundation.Grid.RefElements.Line ] 2 ilPSP.Partitioning ilPSP.Partitioning 0 1 ilPSP.Partitioning BoSSS.Foundation.Grid.Classic.GridData

In [11]:
myDb.SaveGrid(ref grid);
c.SetGrid(grid);

Specification of boundary conditions with a parabolic velocity profile for the inlet

In [12]:
c.BoundaryValues.Clear();
c.AddBoundaryValue("Velocity_Inlet", "VelocityX", 
      (X => (4.1 * 1.5 * (X[1] + 2) * (4.1 - (X[1] + 2)) / (4.1 * 4.1))));

*Fluid Properties*

Note: The characteristic length and fluid density are choosen to one. 
Therefore, the viscosity can be defined by $\frac{1}{reynolds}$.

In [13]:

double reynolds            = 20;
c.PhysicalParameters.rho_A = 1;
c.PhysicalParameters.mu_A  = 1.0/reynolds;

Bool parameter whether the Navier-Stokes or Stokes equations
should be solved

In [14]:
c.PhysicalParameters.IncludeConvection = true;

Initial Values are set to 0; Note that the following lines are only for 
demonstration -- if no initial value is specified, 0 is set automatically.

In [15]:
c.InitialValues.Clear();
c.InitialValues.Add("VelocityX", new Formula("X => 0.0", false));
c.InitialValues.Add("VelocityY", new Formula("X => 0.0", false));
c.InitialValues.Add("Pressure", new Formula("X => 0.0", false));

Timestepping properties:
Most solvers in **BoSSS** simulate transient equations. Configuring
a steady simulation confiures one very large timestep.

In [16]:
c.TimesteppingMode = AppControl._TimesteppingMode.Steady;

## Run a simulation

The solver can be run inline (i.e. within the *BoSSSpad* process) by 
executing the `Run` method on the control objece `c`.
An inline run will block *BoSSSpad* until the solver exits.

In [ ]:
c.Run();

Session ID: 16ab08ae-014b-4145-ada8-012af6788134, DB path: 'C:\Users\flori\AppData\Local\Temp\1950674969'.
Level-Set field Phi is **exactly** zero: setting entire field to -1.
Starting time step 1, dt = 1.7976931348623158E+304 ...
 #Line, #Time, #Iter  #Spc	L2Norm MomentumX    L2Norm MomentumY    L2Norm ContiEq      L2Norm Total        
     1,     1,     1     A	1.000837E+004       0.000000E+000       6.317042E+000       1.000837E+004       
                         B	0.000000E+000       0.000000E+000       0.000000E+000       0.000000E+000       
     2,     1,     2     A	6.372073E+000       1.261499E-003       1.766295E-013       6.372073E+000       
                         B	0.000000E+000       0.000000E+000       0.000000E+000       0.000000E+000       


High residual from direct solver (using ilPSP.LinSolvers.MUMPS.MUMPSSolver).
    L2 Norm of RHS:         0.04480812310423826
    L2 Norm of Solution:    0.07010957002391517
    L2 Norm of Residual:    0.00012363493088295293
    Relative Residual norm: 2.689853796382119E-05
    Matrix Inf norm:        4.596343899777868



     3,     1,     3     A	8.291668E-003       1.465587E-003       8.146234E-007       8.420196E-003       
                         B	0.000000E+000       0.000000E+000       0.000000E+000       0.000000E+000       


High residual from direct solver (using ilPSP.LinSolvers.MUMPS.MUMPSSolver).
    L2 Norm of RHS:         0.00021972129706530322
    L2 Norm of Solution:    0.0726408486092661
    L2 Norm of Residual:    4.475747573231172E-05
    Relative Residual norm: 9.737625536739794E-06
    Matrix Inf norm:        4.596343899592666



     4,     1,     4     A	1.783493E-003       8.823600E-004       4.422456E-007       1.989826E-003       
                         B	0.000000E+000       0.000000E+000       0.000000E+000       0.000000E+000       


...

High residual from direct solver (using ilPSP.LinSolvers.MUMPS.MUMPSSolver).
    L2 Norm of RHS:         9378971590.713938
    L2 Norm of Solution:    19871268450208.418
    L2 Norm of Residual:    9165623175586.195
    Relative Residual norm: 0.46125003034167467
    Matrix Inf norm:        55.3754340909281



    24,     1,    24     A	1.031532E+020       3.159116E+019       6.663222E+009       1.078823E+020       
                         B	0.000000E+000       0.000000E+000       0.000000E+000       0.000000E+000       


High residual from direct solver (using ilPSP.LinSolvers.MUMPS.MUMPSSolver).
    L2 Norm of RHS:         671516421746880.1
    L2 Norm of Solution:    1.5707987826578104E+18
    L2 Norm of Residual:    9.728815854633139E+17
    Relative Residual norm: 0.6193546851476334
    Matrix Inf norm:        30.76409861813698



    25,     1,    25     A	6.321838E+026       2.336840E+026       2.064206E+013       6.739915E+026       
                         B	0.000000E+000       0.000000E+000       0.000000E+000       0.000000E+000       


High residual from direct solver (using ilPSP.LinSolvers.MUMPS.MUMPSSolver).
    L2 Norm of RHS:         8.166433881145303E+19
    L2 Norm of Solution:    2.041757889439647E+23
    L2 Norm of Residual:    1.293261944188354E+23
    Relative Residual norm: 0.6334061207145795
    Matrix Inf norm:        97.53475806005555



## Postprocessing

In order to postprocess data we need to export it with the following command. This creates a folder containing the data as files (In Jupyter you need to copy the commands from the markdown into a code field to execute them)

```csharp
myDb.Sessions.First().Export().Do();
```

You can now go to the path and Plot the data using a programm you prefer (e.g. VisIt or Paraview)


Open the ExportDirectory to view the *.plt files (does only work in BoSSSPad, but you should see the path from the commands before)
```csharp
myDb.Sessions.First().OpenExportDirectory();
```
Some information like the console output or a log containing various physical values can be found in the session directory (does only work in BoSSSPad, but you should see the path from the commands before)
```csharp
myDb.Sessions.First().OpenSessionDirectory();
```
Delete database
```csharp
DatabaseUtils.DeleteDatabase(myDb.Path); 
```

## Immersed boundary method

It is also possible to use the immersed boundary feature of our incompressible Navier-Stokes Solver.

For this example we have to change two parts of the code: First, for a good result, we have to refine 
the grid at the position of the cylinder.


x-Direction (using also hyperbolic tangential distribution)
```csharp
var _xNodes1 = Grid1D.TanhSpacing(-2, -1, 10, 0.5, false); 
_xNodes1     = _xNodes1.GetSubVector(0, (_xNodes1.Length - 1));
var _xNodes2 = GenericBlas.Linspace(-1, 2, 35); 
_xNodes2     = _xNodes2.GetSubVector(0, (_xNodes2.Length - 1));
var _xNodes3 = Grid1D.TanhSpacing(2, 20, 60 , 1.5, true);  
var xNodes   = ArrayTools.Cat(_xNodes1, _xNodes2, _xNodes3);
```

y-Direction
```csharp
var _yNodes1 = Grid1D.TanhSpacing(-2, -1, 7, 0.9, false); 
_yNodes1     = _yNodes1.GetSubVector(0, (_yNodes1.Length - 1));
var _yNodes2 = GenericBlas.Linspace(-1, 1, 25); 
_yNodes2     = _yNodes2.GetSubVector(0, (_yNodes2.Length - 1));
var _yNodes3 = Grid1D.TanhSpacing(1, 2.1, 7, 1.1, true);  
var yNodes   = ArrayTools.Cat(_yNodes1, _yNodes2, _yNodes3);
```

Furthermore, the cylinder immersing the fluid should be described
by using the zero contour of a level set function. The radius of the cylinder is set to 0.5.

```csharp
c.InitialValues.Add("Phi", new Formula("X => -(X[0]).Pow2() + -(X[1]).Pow2() + 0.25", false));
```

Example control files for both,
the channel and the flow around a cylinder can be found in the ControlExample directory. As soon as we run 
the simulation again we can take a look at the plots and the PhysicalData file in the session directory. 
There we can find for example lift and drag forces acting on the cylinder.